In [2]:
import requests
import pandas as pd

def fetch_paginated_api(url, token, params=None, page_param="page", limit_param="limit", 
                        limit=100, max_pages=None, as_dataframe=True):
    """
    Fetch paginated data from a Bearer-token protected API endpoint.

    Args:
        url (str): The API endpoint URL.
        token (str): Your Bearer token.
        params (dict, optional): Extra query parameters.
        page_param (str): The name of the 'page' parameter in the API (default: 'page').
        limit_param (str): The name of the 'limit' parameter in the API (default: 'limit').
        limit (int): Number of records per page (default: 100).
        max_pages (int, optional): Maximum number of pages to fetch (for safety).
        as_dataframe (bool): Return as pandas DataFrame if True, else return raw JSON.

    Returns:
        pd.DataFrame or list: Combined results from all pages.
    """
    headers = {"Authorization": f"Bearer {token}"}
    all_data = []
    page = 1

    while True:
        query_params = params.copy() if params else {}
        query_params.update({page_param: page, limit_param: limit})
        
        response = requests.get(url, headers=headers, params=query_params)
        
        if response.status_code != 200:
            raise Exception(f"Error {response.status_code}: {response.text}")

        data = response.json()

        # Handle list or dict response
        if isinstance(data, dict):
            # Look for list-like values inside the dict
            for v in data.values():
                if isinstance(v, list):
                    chunk = v
                    break
            else:
                chunk = [data]
        else:
            chunk = data

        if not chunk:  # stop if no data returned
            break

        all_data.extend(chunk)
        page += 1

        if max_pages and page > max_pages:
            break

    if as_dataframe:
        return pd.DataFrame(all_data)
    return all_data


In [27]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

API_TOKEN = os.getenv("API_TOKEN")
API_URL = os.getenv("API_URL")
endpoint_billing_plans = "billing/plans"
endpoint_billing_subscriptions = "billing/subscriptions"


In [28]:
url=f"{API_URL.rstrip('/')}/{endpoint_billing_plans.lstrip('/')}"
print(url)

https://api-m.sandbox.paypal.com/v1/billing/plans


In [29]:
df_billing_plans = fetch_paginated_api(
    url=f"{API_URL.rstrip('/')}/{endpoint_billing_plans.lstrip('/')}",
    token=API_TOKEN,
    limit=20,
    max_pages=20   # safety cap
)
print(df_billing_plans.shape)

(28, 8)


In [8]:
df_billing_plans

,id,product_id,name,status,description,usage_type,create_time,links
0,P-2FT022935G912211FM6NCCGA,PROD-14R35346HN469743F,Video Streaming Service Plan,ACTIVE,Video Streaming Service basic plan,LICENSED,2025-01-29T12:37:44Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...
1,P-84G57112F2188035DM6NCWQA,PROD-688529992L681534R,Lebenslauf,ACTIVE,Das weiss doch vorher niemand,LICENSED,2025-01-29T13:21:04Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...
2,P-76S59504HH513745WM6NDM6A,PROD-0DP6039620958130N,Dolomiten,ACTIVE,Das weiss doch vorher niemand,LICENSED,2025-01-29T14:08:56Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...
3,P-17B61707G0673854KM6NUXHY,PROD-9A413705KL335452R,WUH,ACTIVE,Jahresabo,LICENSED,2025-01-30T09:51:27Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...
4,P-5JG56959P6399794RM6NZ6ZA,PROD-34Y162077A826044N,Donnerstag Nachmittag,ACTIVE,Das weiss doch vorher niemand,LICENSED,2025-01-30T15:48:52Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...
5,P-31560533R81481050M7L6T2A,PROD-2HJ69743CN927261X,FISCH & FANG Abo,ACTIVE,NaN,LICENSED,2025-03-17T09:22:48Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...
6,P-92D38243N40194158M7NJPXY,PROD-6TR13458AT717121C,WILD UND HUND Abo,ACTIVE,NaN,LICENSED,2025-03-19T10:09:35Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...
7,P-5VL190622N512041LM7NMTBY,PROD-3Y7929373F111070T,FISCH & FANG Abo,ACTIVE,NaN,LICENSED,2025-03-19T13:41:27Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...
8,P-79G57279YY868574UM7NMXHI,PROD-3Y7929373F111070T,FISCH & FANG Abo,ACTIVE,NaN,LICENSED,2025-03-19T13:50:21Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...
9,P-68G5901538848992RM7OP2VI,PROD-4K145476CP508805P,WILD UND HUND Digital-Abo,ACTIVE,NaN,LICENSED,2025-03-21T05:47:01Z,[{'href': 'https://api.sandbox.paypal.com/v1/b...


In [ ]:
df_subscriptions_list = fetch_paginated_api(
    url=f"{API_URL.rstrip('/')}/{endpoint_billing_subscriptions.lstrip('/')}",
    token=API_TOKEN,
    limit=20,
    max_pages=20   # safety cap
)
print(df_subscriptions_list.shape)

(39, 5)


In [10]:
df_subscriptions_list

,status,id,auto_renewal,create_time,links
0,ACTIVE,I-A2FY2FL6TKSA,False,2025-01-29T13:15:51Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...
1,ACTIVE,I-TMYXN1B7XMTP,False,2025-01-29T13:24:06Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...
2,ACTIVE,I-6XUPX2YKRFSM,False,2025-01-29T14:10:45Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...
3,ACTIVE,I-KTVGS6MDW2U1,False,2025-01-30T15:52:16Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...
4,ACTIVE,I-APY0MWDJGSWV,False,2025-03-17T09:23:17Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...
5,ACTIVE,I-GT7G5LKER1L9,False,2025-03-17T12:08:46Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...
6,ACTIVE,I-FB52YNGBSGL7,False,2025-03-19T10:11:04Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...
7,ACTIVE,I-T2HVPTM2EM1T,False,2025-03-19T13:41:46Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...
8,ACTIVE,I-PGVH4PHR605S,False,2025-03-19T13:50:47Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...
9,ACTIVE,I-SHERR78THNCK,False,2025-03-19T15:37:55Z,[{'href': 'https://api-m.sandbox.paypal.com/v1...


In [ ]:
import oracledb

# Connect using username/password
connection = oracledb.connect(
    user="your_username",
    password="your_password",
    dsn="host:port/service_name"  # e.g. "localhost:1521/XEPDB1"
)

cursor = connection.cursor()
cursor.execute("SELECT sysdate FROM dual")

for row in cursor:
    print(row)

cursor.close()
connection.close()

In [ ]:
df_cover = pd.read_sql("SELECT * FROM employees WHERE ROWNUM <= 10", con=connection)
df_cover.head()